In [1]:
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import urllib.request
import zipfile
import io

# ETL

In [86]:
class ETLPipeline:
    """Class untuk mengotomatisasi proses ETL"""
    
    def __init__(self, db_path='brazil_stock_market.db'):
        """Inisialisasi pipeline"""
        self.conn = duckdb.connect(db_path)
        self.initialized = False
        self.log_table_setup()
    
    def log_table_setup(self):
        """Membuat tabel log untuk melacak proses ETL"""
        self.conn.execute("""
        CREATE TABLE IF NOT EXISTS "etlLog" (
            "logId" INTEGER PRIMARY KEY,
            "processName" VARCHAR,
            "startTime" TIMESTAMP,
            "endTime" TIMESTAMP,
            "recordsProcessed" INTEGER,
            "status" VARCHAR,
            "message" VARCHAR
        )
        """)
    
    def log_process_start(self, process_name):
        """Mencatat mulainya proses ETL"""
        log_id = self.conn.execute('SELECT COALESCE(MAX(logId), 0) + 1 FROM "etlLog"').fetchone()[0]
        self.conn.execute("""
        INSERT INTO "etlLog" (logId, processName, startTime, status)
        VALUES (?, ?, CURRENT_TIMESTAMP, 'RUNNING')
        """, [log_id, process_name])
        return log_id
    
    def log_process_end(self, log_id, records=0, status='SUCCESS', message=None):
        """Mencatat selesainya proses ETL"""
        self.conn.execute("""
        UPDATE "etlLog"
        SET endTime = CURRENT_TIMESTAMP,
            recordsProcessed = ?,
            status = ?,
            message = ?
        WHERE logId = ?
        """, [records, status, message, log_id])

    def ensure_common_columns(self):
        """
        Ensure all target tables have 'effectiveDate', 'expirationDate', and 'currentFlag' columns.
        """
        columns_to_add = [
            ("effectiveDate", "DATE"),
            ("expirationDate", "DATE"),
            ("currentFlag", "BOOLEAN")
        ]

        target_tables = ["dimCoin", "dimCompany", "dimTime", "factCoins", "factStocks"]

        for table in target_tables:
            for column_name, column_type in columns_to_add:
                alter_sql = f"""
                ALTER TABLE {table}
                ADD COLUMN IF NOT EXISTS {column_name} {column_type}
                """
                self.conn.execute(alter_sql)
                
    def initialize_warehouse(self):
        """Inisialisasi skema data warehouse jika belum ada"""
        if self.initialized:
            return
        
        log_id = self.log_process_start("initialize_warehouse")
        
        try:
            # Dimensi Company
            self.conn.execute("""
            CREATE TABLE IF NOT EXISTS "dimCompany" (
                "keyCompany" INT NOT NULL,
                "stockCodeCompany" VARCHAR(32) NOT NULL,
                "nameCompany" VARCHAR(64) NOT NULL,
                "sectorCodeCompany" VARCHAR(32) NOT NULL,
                "sectorCompany" VARCHAR(256) NOT NULL,
                "segmentCompany" VARCHAR(256) NOT NULL,
                "startedAt" TIMESTAMP NOT NULL DEFAULT NOW(),
                "endedAt" TIMESTAMP NULL,
                "isActive" BOOLEAN NOT NULL DEFAULT TRUE,
                CONSTRAINT companyPK PRIMARY KEY ("keyCompany")
            );
            """)
            
            # Dimensi Coin
            self.conn.execute("""
            CREATE TABLE IF NOT EXISTS "dimCoin" (
                "keyCoin" INT NOT NULL,
                "abbrevCoin" VARCHAR(32) NOT NULL,
                "nameCoin" VARCHAR(32) NOT NULL,
                "symbolCoin" VARCHAR(8) NOT NULL,
                
                CONSTRAINT coinPK PRIMARY KEY (keyCoin)
            );
            """)
            
            # Dimensi Time
            self.conn.execute("""
            CREATE TABLE IF NOT EXISTS "dimTime" (
                "keyTime" INT NOT NULL,
                "datetime" VARCHAR(32) NOT NULL,
                "dayTime" SMALLINT NOT NULL,
                "dayWeekTime" SMALLINT NOT NULL,
                "dayWeekAbbrevTime" VARCHAR(32) NOT NULL,
                "dayWeekCompleteTime" VARCHAR(32) NOT NULL,
                "monthTime" SMALLINT NOT NULL,
                "monthAbbrevTime" VARCHAR(32) NOT NULL,
                "monthCompleteTime" VARCHAR(32) NOT NULL,
                "bimonthTime" SMALLINT NOT NULL,
                "quarterTime" SMALLINT NOT NULL,
                "semesterTime" SMALLINT NOT NULL,
                "yearTime" INT NOT NULL,
                
                CONSTRAINT timePK PRIMARY KEY ("keyTime")
            );
            """)
            
            # Tabel fakta Coins
            self.conn.execute("""
            CREATE TABLE IF NOT EXISTS "factCoins" (
                "keyTime" INT NOT NULL,
                "keyCoin" INT NOT NULL,
                "valueCoin" FLOAT NOT NULL,
                
                FOREIGN KEY (keyTime) REFERENCES dimTime(keyTime),
                FOREIGN KEY (keyCoin) REFERENCES dimCoin(keyCoin),
                CONSTRAINT coinsPK PRIMARY KEY(keyTime, keyCoin)
            );
            """)
            
            # Tabel fakta Stocks
            self.conn.execute("""
            CREATE TABLE IF NOT EXISTS "factStocks" (
                "keyTime" INT NOT NULL,
                "keyCompany" INT NOT NULL,
                "openValueStock" FLOAT NOT NULL,
                "closeValueStock" FLOAT NOT NULL,
                "highValueStock" FLOAT NOT NULL,
                "lowValueStock" FLOAT NOT NULL,
                "quantityStock" FLOAT NOT NULL,
                
                FOREIGN KEY (keyTime) REFERENCES dimTime(keyTime),
                FOREIGN KEY (keyCompany) REFERENCES dimCompany(keyCompany),
                CONSTRAINT stocksPK PRIMARY KEY(keyTime, keyCompany)
            );
            """)
            
            # Menambahkan column SCD
            self.ensure_common_columns()
            
            # Berhasil inisialisasi
            self.initialized = True
            self.log_process_end(log_id, status='SUCCESS', message='Data warehouse schema initialized')
            
        except Exception as e:
            self.log_process_end(log_id, status='ERROR', message=str(e))
            raise
    
    def extract_all_sources(self):
        """Ekstrak data dari semua sumber"""
        log_id = self.log_process_start("extract_all_sources")
        
        try:
            # Bersihkan tabel staging
            self.conn.execute("DROP TABLE IF EXISTS stagingCoin")
            self.conn.execute("DROP TABLE IF EXISTS stagingCompany")
            self.conn.execute("DROP TABLE IF EXISTS stagingTime")
            self.conn.execute("DROP TABLE IF EXISTS stagingCoinValue")
            self.conn.execute("DROP TABLE IF EXISTS stagingStockValue")
            
            # Ekstrak data dari file CSV
            # Create staging tables from CSVs
            self.conn.execute("CREATE TABLE stagingCoin AS SELECT * FROM read_csv_auto('data/dimCoin.csv')")
            self.conn.execute("CREATE TABLE stagingCompany AS SELECT * FROM read_csv_auto('data/dimCompany.csv')")
            self.conn.execute("CREATE TABLE stagingTime AS SELECT * FROM read_csv_auto('data/dimTime.csv')")
            self.conn.execute("CREATE TABLE stagingCoinValue AS SELECT * FROM read_csv_auto('data/factCoins.csv')")
            self.conn.execute("CREATE TABLE stagingStockValue AS SELECT * FROM read_csv_auto('data/factStocks.csv')")
            
            # Hitung total jumlah catatan
            total_records = 0
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingCoin").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingCompany").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingTime").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingCoinValue").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingStockValue").fetchone()[0]
            
            self.log_process_end(log_id, records=total_records, status='SUCCESS')
            return True
            
        except Exception as e:
            self.log_process_end(log_id, status='ERROR', message=str(e))
            raise
    
    def transform_data(self):
        """Transformasi data dari tabel staging"""
        log_id = self.log_process_start("transform_data")
        
        try:
            # 1. Transformasi dimensi Coin
            self.conn.execute("""
            CREATE OR REPLACE TABLE stagingDimCoin AS
            SELECT
                keyCoin,
                abbrevCoin,
                nameCoin,
                symbolCoin,
                '2022-01-01'::DATE AS effectiveDate,
                NULL::DATE AS expirationDate,
                TRUE AS currentFlag
            FROM stagingCoin
            """)

            # 2. Transformasi dimensi Company
            self.conn.execute("""
            CREATE OR REPLACE TABLE stagingDimCompany AS
            SELECT
                keyCompany,
                stockCodeCompany,
                nameCompany,
                sectorCodeCompany,
                sectorCompany,
                segmentCompany,
                NOW() startedAt,
                NULL endedAt,
                TRUE isActive,
                '2022-01-01'::DATE effectiveDate,
                NULL::DATE expirationDate,
                TRUE currentFlag
            FROM stagingCompany
            """)

            # 3. Transformasi dimensi Time dengan translasi nama hari & bulan ke Bahasa Inggris
            self.conn.execute("""
            CREATE OR REPLACE TABLE stagingDimTime AS
            SELECT
                keyTime,
                datetime,
                dayTime,
                dayWeekTime,

                CASE dayWeekAbbrevTime
                    WHEN 'SEG' THEN 'MON'
                    WHEN 'TER' THEN 'TUE'
                    WHEN 'QUA' THEN 'WED'
                    WHEN 'QUI' THEN 'THU'
                    WHEN 'SEX' THEN 'FRI'
                    WHEN 'SAB' THEN 'SAT'
                    WHEN 'DOM' THEN 'SUN'
                    ELSE dayWeekAbbrevTime
                END dayWeekAbbrevTime,

                CASE dayWeekCompleteTime
                    WHEN 'SEGUNDA' THEN 'MONDAY'
                    WHEN 'TERCA' THEN 'TUESDAY'
                    WHEN 'QUARTA' THEN 'WEDNESDAY'
                    WHEN 'QUINTA' THEN 'THURSDAY'
                    WHEN 'SEXTA' THEN 'FRIDAY'
                    WHEN 'SABADO' THEN 'SATURDAY'
                    WHEN 'DOMINGO' THEN 'SUNDAY'
                    ELSE dayWeekCompleteTime
                END dayWeekCompleteTime,
                
                monthTime,

                CASE monthAbbrevTime
                    WHEN 'JAN' THEN 'JAN'
                    WHEN 'FEV' THEN 'FEB'
                    WHEN 'MAR' THEN 'MAR'
                    WHEN 'ABR' THEN 'APR'
                    WHEN 'MAI' THEN 'MAY'
                    WHEN 'JUN' THEN 'JUN'
                    WHEN 'JUL' THEN 'JUL'
                    WHEN 'AGO' THEN 'AUG'
                    WHEN 'SET' THEN 'SEP'
                    WHEN 'OUT' THEN 'OCT'
                    WHEN 'NOV' THEN 'NOV'
                    WHEN 'DEZ' THEN 'DEC'
                    ELSE monthAbbrevTime
                END monthAbbrevTime,
                
                CASE monthCompleteTime
                    WHEN 'JANEIRO' THEN 'JANUARY'
                    WHEN 'FEVEREIRO' THEN 'FEBRUARY'
                    WHEN 'MARCO' THEN 'MARCH'
                    WHEN 'ABRIL' THEN 'APRIL'
                    WHEN 'MAIO' THEN 'MAY'
                    WHEN 'JUNHO' THEN 'JUNE'
                    WHEN 'JULHO' THEN 'JULY'
                    WHEN 'AGOSTO' THEN 'AUGUST'
                    WHEN 'SETEMBRO' THEN 'SEPTEMBER'
                    WHEN 'OUTUBRO' THEN 'OCTOBER'
                    WHEN 'NOVEMBRO' THEN 'NOVEMBER'
                    WHEN 'DEZEMBRO' THEN 'DECEMBER'
                    ELSE monthCompleteTime
                END monthCompleteTime,

                bimonthTime,
                quarterTime,
                semesterTime,
                yearTime,
                '2022-01-01'::DATE effectiveDate,
                NULL::DATE expirationDate,
                TRUE currentFlag
            FROM stagingTime
            """)

            # 4. Transformasi fakta Coins
            self.conn.execute("""
            CREATE OR REPLACE TABLE stagingFactCoins AS
            SELECT
                keyTime,
                keyCoin,
                valueCoin,
                '2022-01-01'::DATE effectiveDate,
                NULL::DATE expirationDate,
                TRUE currentFlag
            FROM stagingCoinValue
            """)

            # 5. Transformasi fakta Stocks
            self.conn.execute("""
            CREATE OR REPLACE TABLE stagingFactStocks AS
            SELECT
                keyTime,
                keyCompany,
                openValueStock,
                closeValueStock,
                highValueStock,
                lowValueStock,
                quantityStock,
                '2022-01-01'::DATE effectiveDate,
                NULL::DATE expirationDate,
                TRUE currentFlag
            FROM stagingStockValue
            """)
            
            # Hitung jumlah catatan
            total_records = 0
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingDimCoin").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingDimCompany").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingDimTime").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingFactCoins").fetchone()[0]
            total_records += self.conn.execute("SELECT COUNT(*) FROM stagingFactStocks").fetchone()[0]

            self.log_process_end(log_id, records=total_records, status='SUCCESS')
            return True
            
        except Exception as e:
            self.log_process_end(log_id, status='ERROR', message=str(e))
            raise
    
    def load_data(self):
        """Load data ke data warehouse"""
        log_id = self.log_process_start("load_data")
        
        try:
            # 1. Muat dimensi Coin
            self.conn.execute("DELETE FROM dimCoin")  # Bersihkan tabel sasaran
            self.conn.execute("INSERT INTO dimCoin SELECT * FROM stagingDimCoin")
            
            # 2. Muat dimensi Company
            self.conn.execute("DELETE FROM dimCompany")  # Bersihkan tabel sasaran
            self.conn.execute("INSERT INTO dimCompany SELECT * FROM stagingDimCompany")
            
            # 3. Muat dimensi Time
            self.conn.execute("DELETE FROM dimTime")  # Bersihkan tabel sasaran
            self.conn.execute("INSERT INTO dimTime SELECT * FROM stagingDimTime")
            
            # 4. Muat fakta Coins dengan pemetaan kunci yang benar
            self.conn.execute("DELETE FROM factCoins")  # Bersihkan tabel sasaran
            self.conn.execute("""
            INSERT INTO factCoins
            SELECT 
                t.keyTime,
                c.keyCoin,
                f.valueCoin,
                f.effectiveDate,
                f.expirationDate,
                f.currentFlag
            FROM stagingFactCoins f
            JOIN dimTime t ON f.keyTime = t.keyTime
            JOIN dimCoin c ON f.keyCoin = c.keyCoin
            """)

            # 5. Muat fakta Stocks dengan pemetaan kunci yang benar
            self.conn.execute("DELETE FROM factStocks")  # Bersihkan tabel sasaran
            self.conn.execute("""
            INSERT INTO factStocks
            SELECT 
                t.keyTime,
                c.keyCompany,
                f.openValueStock,
                f.closeValueStock,
                f.highValueStock,
                f.lowValueStock,
                f.quantityStock,
                f.effectiveDate,
                f.expirationDate,
                f.currentFlag
            FROM stagingFactStocks f
            JOIN dimTime t ON f.keyTime = t.keyTime
            JOIN dimCompany c ON f.keyCompany = c.keyCompany
            """)

            
            # Menghitung jumlah baris yang dimuat
            coin_count = self.conn.execute("SELECT COUNT(*) FROM dimCoin").fetchone()[0]
            company_count = self.conn.execute("SELECT COUNT(*) FROM dimCompany").fetchone()[0]
            time_count = self.conn.execute("SELECT COUNT(*) FROM dimTime").fetchone()[0]
            fact_coin_count = self.conn.execute("SELECT COUNT(*) FROM factCoins").fetchone()[0]
            fact_stock_count = self.conn.execute("SELECT COUNT(*) FROM factStocks").fetchone()[0]

            total_records = coin_count + company_count + time_count + fact_coin_count + fact_stock_count

            self.log_process_end(log_id, records=total_records, status='SUCCESS', 
                message=f'Loaded {coin_count} coins, {company_count} companies, '
                        f'{time_count} time records, {fact_coin_count} coin facts, '
                        f'{fact_stock_count} stock facts')
            return True
            
        except Exception as e:
            self.log_process_end(log_id, status='ERROR', message=str(e))
            raise
    
    def run_full_pipeline(self):
        """Jalankan seluruh pipeline ETL"""
        print("Menjalankan ETL Pipeline lengkap...")
        
        start_time = datetime.now()
        
        try:
            # 1. Inisialisasi warehouse
            self.initialize_warehouse()
            print("✓“ Data warehouse diinisialisasi")
            
            # 2. Ekstrak data
            self.extract_all_sources()
            print("✓“ Data diekstrak dari semua sumber")
            
            # 3. Transform data
            self.transform_data()
            print("✓“ Data ditransformasi")
            
            # 4. Load data
            self.load_data()
            print("✓“ Data dimuat ke data warehouse")
            
            end_time = datetime.now()
            duration = (end_time - start_time).total_seconds()
            
            print(f"\nPipeline ETL selesai dalam {duration:.2f} detik")
            
            # Menampilkan statistik
            stats = self.conn.execute("""
                SELECT 
                    (SELECT COUNT(*) FROM dimCoin) as coin_count,
                    (SELECT COUNT(*) FROM dimCompany) as company_count,
                    (SELECT COUNT(*) FROM dimTime) as time_count,
                    (SELECT COUNT(*) FROM factCoins) as fact_coin_count,
                    (SELECT COUNT(*) FROM factStocks) as fact_stock_count
            """).fetchone()

            print("\nStatistik Data Warehouse:")
            print(f"✓ Dimensi Coin: {stats[0]} baris")
            print(f"✓ Dimensi Company: {stats[1]} baris")
            print(f"✓ Dimensi Time: {stats[2]} baris")
            print(f"✓ Fakta Coin: {stats[3]} baris")
            print(f"✓ Fakta Stock: {stats[4]} baris")
            
            return True
            
        except Exception as e:
            print(f"ERROR: Pipeline ETL gagal: {str(e)}")
            return False
    
    def get_etl_log(self, limit=10):
        """Menampilkan log ETL terakhir"""
        return self.conn.execute(f"""
            SELECT 
                logId,
                processName,
                startTime,
                endTime,
                EXTRACT(EPOCH FROM (endTime - startTime)) as duration_seconds,
                recordsProcessed,
                status,
                message
            FROM etlLog
            ORDER BY logId DESC
            LIMIT {limit}
        """).fetchdf()
    
    def __del__(self):
        """Menutup koneksi saat objek dihapus"""
        try:
            if hasattr(self, 'conn'):
                self.conn.close()
                print("Koneksi DuckDB ditutup.")
        except:
            pass

In [88]:
pipeline = ETLPipeline()
pipeline.run_full_pipeline()

Koneksi DuckDB ditutup.
Menjalankan ETL Pipeline lengkap...
✓“ Data warehouse diinisialisasi
✓“ Data diekstrak dari semua sumber
✓“ Data ditransformasi
✓“ Data dimuat ke data warehouse

Pipeline ETL selesai dalam 6.84 detik

Statistik Data Warehouse:
✓ Dimensi Coin: 2 baris
✓ Dimensi Company: 607 baris
✓ Dimensi Time: 9680 baris
✓ Fakta Coin: 13300 baris
✓ Fakta Stock: 680150 baris


True

In [43]:
pipeline.get_etl_log()

,logId,processName,startTime,endTime,duration_seconds,recordsProcessed,status,message
0,9,transform_data,2025-03-20 22:26:13.063,2025-03-20 22:26:13.416,0.353,703741,SUCCESS,None
1,8,extract_all_sources,2025-03-20 22:26:12.219,2025-03-20 22:26:13.058,0.839,703741,SUCCESS,None
2,7,initialize_warehouse,2025-03-20 22:26:12.202,2025-03-20 22:26:12.216,0.014,0,SUCCESS,Data warehouse schema initialized
3,6,transform_data,2025-03-20 22:25:18.547,2025-03-20 22:25:18.888,0.341,703741,SUCCESS,None
4,5,extract_all_sources,2025-03-20 22:25:17.688,2025-03-20 22:25:18.543,0.855,703741,SUCCESS,None
5,4,initialize_warehouse,2025-03-20 22:25:17.671,2025-03-20 22:25:17.685,0.014,0,SUCCESS,Data warehouse schema initialized
6,3,transform_data,2025-03-20 22:20:10.415,2025-03-20 22:20:10.420,0.005,0,ERROR,"Binder Error: Referenced column ""coin_id"" not ..."
7,2,extract_all_sources,2025-03-20 22:20:09.548,2025-03-20 22:20:10.411,0.863,703741,SUCCESS,None
8,1,initialize_warehouse,2025-03-20 22:20:09.534,2025-03-20 22:20:09.545,0.011,0,SUCCESS,Data warehouse schema initialized


# Debug

In [30]:
# Connect to DuckDB (in-memory or file)
con = duckdb.connect('brazil_stock_market.db')  # or ':memory:' for in-memory DB

## Debug Table

In [ ]:
con.execute("SELECT * FROM stagingdimTime LIMIT 10").df()

In [ ]:
info = con.execute("PRAGMA table_info('stagingdimTime')").fetchall()
for col in info:
    print(f"{col[1]}: {col[2]}")

print()

info = con.execute("PRAGMA table_info('dimTime')").fetchall()
for col in info:
    print(f"{col[1]}: {col[2]}")

## Debug show tables

In [29]:
# Show all tables
tables = con.execute("SHOW TABLES").fetchall()

# Print the list of tables
for table in tables:
    print(table[0])

dimCoin
dimCompany
dimTime
etlLog
factCoins
factStocks
stagingCoin
stagingCoinValue
stagingCompany
stagingStockValue
stagingTime


## Debug delete all tables

In [87]:
con.execute(f"DROP TABLE IF EXISTS factCoins")
con.execute(f"DROP TABLE IF EXISTS factStocks")

# Get list of all tables in the 'main' schema
tables = con.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main'
""").fetchall()

# Drop each table
for table in tables:
    table_name = table[0]
    con.execute(f"DROP TABLE IF EXISTS {table_name}")

print("All tables deleted.")


All tables deleted.
